In [11]:
import os
import timm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PIL import Image

In [12]:
train_dir = "G:/v2/Nail Disease DataSet/Train"
valid_dir = "G:/v2/Nail Disease DataSet/Test"

# Check if the paths exist to avoid errors
assert os.path.exists(train_dir), f"Training directory not found at: {train_dir}"
assert os.path.exists(valid_dir), f"Validation directory not found at: {valid_dir}"

print('Dataset paths set.')

Dataset paths set.


In [13]:
# Define the transformations for the train and validation data
train_transforms = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.ColorJitter(brightness=0.125, contrast=0.125, saturation=0.05, hue=0.025),
    transforms.RandomAffine(degrees=10, translate=(0.075, 0.075), shear=0.025),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize((192, 192)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Load datasets using ImageFolder
train_data = ImageFolder(root=train_dir, transform=train_transforms)
valid_data = ImageFolder(root=valid_dir, transform=valid_transforms)

# DataLoader for train and validation
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

print(f'Found {len(train_data.classes)} classes.')
print(f'Training samples: {len(train_data)}, Validation samples: {len(valid_data)}')


Found 17 classes.
Training samples: 655, Validation samples: 183


In [14]:
# Load the ResNet18 model from timm
model = timm.create_model('resnet18d', pretrained=True, num_classes=len(train_data.classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3.25e-4, weight_decay=1e-3)

print('Model, criterion, and optimizer are ready.')

Model, criterion, and optimizer are ready.


In [15]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=16):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Training on device: {device}")

    for epoch in range(epochs):
        # --- Training Phase ---
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        train_loss = running_loss / len(train_loader)

        # --- Validation Phase ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_loss_avg = val_loss / len(valid_loader)

        print(f'Epoch {epoch + 1}/{epochs} | '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}% | '
              f'Val Loss: {val_loss_avg:.4f}, Val Accuracy: {val_accuracy:.2f}%')

In [16]:
print("Starting training...")
train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=30)
print("Training finished.")


Starting training...
Training on device: cpu
Epoch 1/30 | Train Loss: 2.7106, Train Accuracy: 16.49% | Val Loss: 2.5673, Val Accuracy: 19.67%
Epoch 2/30 | Train Loss: 2.1574, Train Accuracy: 40.00% | Val Loss: 1.9800, Val Accuracy: 39.89%
Epoch 3/30 | Train Loss: 1.6263, Train Accuracy: 58.47% | Val Loss: 1.3780, Val Accuracy: 60.11%
Epoch 4/30 | Train Loss: 1.1346, Train Accuracy: 69.77% | Val Loss: 0.9174, Val Accuracy: 77.05%
Epoch 5/30 | Train Loss: 0.8818, Train Accuracy: 77.71% | Val Loss: 0.6638, Val Accuracy: 83.06%
Epoch 6/30 | Train Loss: 0.6547, Train Accuracy: 84.58% | Val Loss: 0.5107, Val Accuracy: 84.70%
Epoch 7/30 | Train Loss: 0.4599, Train Accuracy: 87.79% | Val Loss: 0.2598, Val Accuracy: 95.63%
Epoch 8/30 | Train Loss: 0.3559, Train Accuracy: 92.06% | Val Loss: 0.2376, Val Accuracy: 94.54%
Epoch 9/30 | Train Loss: 0.2971, Train Accuracy: 92.98% | Val Loss: 0.1788, Val Accuracy: 96.72%
Epoch 10/30 | Train Loss: 0.2345, Train Accuracy: 96.03% | Val Loss: 0.1448, Val A

In [17]:
print("finished")

finished


In [18]:
# Save the trained model
model_save_path = "17classes_resnet_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to 17classes_resnet_model.pth


In [ ]:
# Starting training...
# Training on device: cpu
# Epoch 1/30 | Train Loss: 2.7106, Train Accuracy: 16.49% | Val Loss: 2.5673, Val Accuracy: 19.67%
# Epoch 2/30 | Train Loss: 2.1574, Train Accuracy: 40.00% | Val Loss: 1.9800, Val Accuracy: 39.89%
# Epoch 3/30 | Train Loss: 1.6263, Train Accuracy: 58.47% | Val Loss: 1.3780, Val Accuracy: 60.11%
# Epoch 4/30 | Train Loss: 1.1346, Train Accuracy: 69.77% | Val Loss: 0.9174, Val Accuracy: 77.05%
# Epoch 5/30 | Train Loss: 0.8818, Train Accuracy: 77.71% | Val Loss: 0.6638, Val Accuracy: 83.06%
# Epoch 6/30 | Train Loss: 0.6547, Train Accuracy: 84.58% | Val Loss: 0.5107, Val Accuracy: 84.70%
# Epoch 7/30 | Train Loss: 0.4599, Train Accuracy: 87.79% | Val Loss: 0.2598, Val Accuracy: 95.63%
# Epoch 8/30 | Train Loss: 0.3559, Train Accuracy: 92.06% | Val Loss: 0.2376, Val Accuracy: 94.54%
# Epoch 9/30 | Train Loss: 0.2971, Train Accuracy: 92.98% | Val Loss: 0.1788, Val Accuracy: 96.72%
# Epoch 10/30 | Train Loss: 0.2345, Train Accuracy: 96.03% | Val Loss: 0.1448, Val Accuracy: 96.72%
# Epoch 11/30 | Train Loss: 0.2010, Train Accuracy: 96.18% | Val Loss: 0.0931, Val Accuracy: 97.27%
# Epoch 12/30 | Train Loss: 0.1338, Train Accuracy: 97.25% | Val Loss: 0.1219, Val Accuracy: 96.17%
# Epoch 13/30 | Train Loss: 0.1512, Train Accuracy: 96.18% | Val Loss: 0.0878, Val Accuracy: 98.91%
# Epoch 14/30 | Train Loss: 0.1474, Train Accuracy: 97.56% | Val Loss: 0.0571, Val Accuracy: 98.91%
# Epoch 15/30 | Train Loss: 0.1082, Train Accuracy: 97.40% | Val Loss: 0.0614, Val Accuracy: 98.36%
# Epoch 16/30 | Train Loss: 0.0722, Train Accuracy: 98.63% | Val Loss: 0.0348, Val Accuracy: 98.91%
# Epoch 17/30 | Train Loss: 0.0810, Train Accuracy: 98.02% | Val Loss: 0.0780, Val Accuracy: 97.81%
# Epoch 18/30 | Train Loss: 0.0776, Train Accuracy: 98.17% | Val Loss: 0.0479, Val Accuracy: 98.36%
# Epoch 19/30 | Train Loss: 0.0792, Train Accuracy: 98.47% | Val Loss: 0.0346, Val Accuracy: 98.91%
# Epoch 20/30 | Train Loss: 0.0565, Train Accuracy: 99.08% | Val Loss: 0.0335, Val Accuracy: 98.36%
# Epoch 21/30 | Train Loss: 0.0561, Train Accuracy: 98.63% | Val Loss: 0.0354, Val Accuracy: 98.36%
# Epoch 22/30 | Train Loss: 0.0737, Train Accuracy: 98.02% | Val Loss: 0.1136, Val Accuracy: 95.63%
# Epoch 23/30 | Train Loss: 0.0715, Train Accuracy: 98.63% | Val Loss: 0.0201, Val Accuracy: 99.45%
# ...
# Epoch 28/30 | Train Loss: 0.0423, Train Accuracy: 98.78% | Val Loss: 0.0159, Val Accuracy: 99.45%
# Epoch 29/30 | Train Loss: 0.0579, Train Accuracy: 98.32% | Val Loss: 0.0217, Val Accuracy: 98.91%
# Epoch 30/30 | Train Loss: 0.0425, Train Accuracy: 98.93% | Val Loss: 0.0120, Val Accuracy: 99.12%
# Training finished.